I was asked to revisit the difference between function declaration and calling, class declaration and instantiation and method declaration and calling.

We use `def` to declare a function. What follows is what we call the _function signature_, then a `:` and then the _function body_ which is indented by four spaces and often ends with a `return` statement:

In [1]:
def square(x):
    return x**2

Using the function signature we can also call the function. The arguments declared in the signature are then used to process and we get back whatever we evaluate after the `return`:

In [2]:
square(2)

4

The `class` keyword introduces a similar syntax with very different meaning. Like for a function we give a name to the class. By convention this name is capitalized.

Anything following in parentheses is not a parameter, but a so called _superclass_ (or _parent class_). In fact, all objects in Python inherit from `object`. The part in parentheses in `class Classname(object)` is usually omitted.

The parameters we call upon _instantiation_ of a class are found in the special method `__init__`. Special methods are methods with leading and trailing double underscores. Python uses those for internal purposes, they can be defined but should not be called directly.

As `__init__` is a method, it has access to a `self` parameter. If we call the class name (_instantiation_) we create a **new** object of this class. The `__init__` method forges this new object into the shape of the class. The class is the blueprint for any new instance of it.

In [3]:
class Powerizer(object):
    def __init__(self, to):
        self.to = to
        
    def power(self, x):
        return x**self.to
    
    def square(self, x):
        return x**2
        
p = Powerizer(to=3)

In the example class we create objects which are _powerizers_ and carry a state in the attribute `to`. Any two instances of this class can carry different state in `to`.

In [4]:
p.to

3

A method is very similar to a function. The key differences are, that methods live inside classes and their first argument is always `self`. This first argument is handed to the method by Python itself. We only ever need to call methods with the parameters following this `self`.

Using `self`, the method has access to the individual state an object carries. For instance, `p` has its `to` attribute set to $3$. The `power` method raises its argument to the power of `to` (in the case of `p` this is the power of $3$).

Notice also the `p.square` method. It is similar to the `square` function in what it does. It would have access to the state of the `Powerizer` instance it is called on, we just chose to ignore this `self`.

In [5]:
p.power(3)

27

# The Zen of Python

As we have seen throughout this semester, there are often many paths to the same result in Python. However, they are rarely equal. An order of elegance and _pythonism_ separates them.

In times of distress we can turn to the _Zen of Python_ for guidance among these paths. You can summon it with

~~~python
import this
~~~

In [6]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


We've already seen namespaces to great extent. Here is the summary:

+ To **look** into a namespace use the `dir(namespace)` function
+ To **move** into a namespace use the dot: `.`
+ Namespaces are available on all Python objects, including modules
+ There is a global namespace which you can discover by calling `dir()` without arguments

In [7]:
import numpy as np

In [8]:
np.square([2,3,4,5])

array([ 4,  9, 16, 25], dtype=int32)

## Identity and equality

Reconsider the last part of the Pre-Exercise 11. In certain cases there certainly could be two distinct fungi at the same position. The description disregards such cases on purpose, of course, as to not complicate the matter. But what if we wanted to know if a fungus merely grows at the same spot or is actually the very same fungus as another?

In other words: How do we compare objects for (in-)equality and identity?

In [9]:
class A:
    pass
a1 = A()
id(a1)

1792440591816

As we can see, even an object of a completely empty class `A` has an identity. This identity can be queried using the `id` function.

In [10]:
a2 = A()
id(a2)

1792440593928

Another object of the same class does not have the same `id`. So surely they are not the same:

In [11]:
a2 is a1

False

But what about objects that can be equal while they are not the same?

Consider the class `Circle` which has a radius. Two circles are equal if the have the same radius.

In [12]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
c1 = Circle(1)
c2 = Circle(1)

iIf we want to compare for equality, rather than identity, we already have learnt to use `==`.

The following should yield `True` given our reasoning:

In [13]:
c1 == c2, c1 is c2

(False, False)

What has happened here?

Behind the scenes Python uses identity as equality if we don't tell it otherwise. Here is how we can define our own equality.

It works very similar to how we defined our `distance` method above:

In [14]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    def __eq__(self, other):
        return self.radius == other.radius
    
c1 = Circle(1)
c2 = Circle(1)
c3 = c2

Now we can distinguish if two circles really **are** the same, or if they merely satisfy our conditions of equality.

This can be especially useful if we want to treat the two circles differently later on.

In [15]:
c1 == c3, \
c1 is c3, \
c2 is c3, \
c2 == c3

(True, False, True, True)

## Mutability and its pitfalls

The issue of identity is especially pressing when mutating objects. Lists e.g. are mutable:

In [16]:
l1 = list()
l2 = l1 # assign l1 to l2
l1.append('a')
# appending on l2 also appends on l1 -> They are the same object!
print(l1)
print(l2)

l1 is l2 # This is the reason they seem to both change

['a']
['a']


True

It may seem intuitive, that if we say `l1 = l2` we get this result. But this does not work if we re-assign rather than mutate. For example with integers:

In [17]:
i = 1
j = i
j += 1
i,j

(1, 2)

or with tuples:

In [18]:
t = (1,2)
u = t
u += (3,)
t,u

((1, 2), (1, 2, 3))

This is why lists being mutable and tuples being immutable is such an important thing to know in Python.

In more obscure use cases like the following, the mutability of lists (and object attributes in Python in general) can catch us off guard:

In [19]:
li = [list()] * 3 # create a list of three empty lists
print(li) # check the result is correct

li[0].append('a') # append `1` to the first list in the super-list
print(li) # what happened?

[[], [], []]
[['a'], ['a'], ['a']]


Now we can also understand better how list comprehensions work:

In [20]:
li = [list() for _ in range(3)]
print(li)

li[0].append('a') # append `1` to the first list in the super-list
print(li) # what happened?

[[], [], []]
[['a'], [], []]


## Multiple inheritance

We now know that a class `B` can inherit methods from a class `A` by defining it as `B(A)`. But what if another class `C` inherits from `B`? Will it also inherit from `A`?

In [21]:
class Animal:
    lifepoints = 63
class Rabbit(Animal):
    pass
class Wolf(Animal):
    pass

wolf1 = Wolf()
wolf1.lifepoints # it does!

63

This was foreseeable. Let's have a look at a more interesting case: *Multiple inheritance*

Let's say we have two separate classes:
+ `Predator` defines an attribute `lifepoints` as `63`
+ `Prey` defined an attribute, also named `lifepoints`, as `42`

The class `Fox` inherits from both `Predator` and `Prey`, just as `Hog` but in reverse order:

In [22]:
class Predator:
    lifepoints = 63
class Prey:
    lifepoints = 42
    
class Fox(Predator, Prey):
    pass
class Hog(Prey, Predator):
    pass

fox1 = Fox()
hog1 = Hog()
fox1.lifepoints, hog1.lifepoints

(63, 42)

We can observe, that the order of inheritance is in fact important!

If we want to have a look at the order at which attribute and method names are looked up, we can use the `__mro__` attribute of a class:

In [23]:
Fox.__mro__

(__main__.Fox, __main__.Predator, __main__.Prey, object)

In [24]:
Hog.__mro__

(__main__.Hog, __main__.Prey, __main__.Predator, object)

Just above I mentioned a method or attribute can be *overridden*. Let's see how that works.

Say in the last example we also want to add a fusion power plant. It is like a normal power plant, but we need to know how much hydrogen it uses:

In [25]:
class PowerPlant:
    def __init__(self, f, out):
        self.f = f
        self.out = out
        self.inoperation = 'no'
        self.shape = '.'
        
    def emissions(self):
        return self.f * self.out
    
    def turn_on(self):
        self.inoperation = 'yes'
        
    def draw(self):
        plt.scatter(rd.randint(0,5), rd.randint(0,5)
                   , marker=self.shape
                   , s = self.out / 10
                   , color = 'red' if self.inoperation == 'no' else 'green'
                   )

class FusionPlant(PowerPlant):
    def __init__(self, hydrogen_demand):
        self.hydrogen_demand = hydrogen_demand
        
fp = FusionPlant(300)
fp.hydrogen_demand

300

That seems to work, but the mindful reader may have noticed: We do not initialize `PowerPlant` proper!

We can observe this easily:

In [26]:
fp.shape, fp.out. fp.f

AttributeError: 'FusionPlant' object has no attribute 'shape'

So how would we do this? We call `super` !

In [27]:
class FusionPlant(PowerPlant):
    def __init__(self, f, out, hydrogen_demand):
        super(FusionPlant, self).__init__(f, out)
        
        self.hydrogen_demand = hydrogen_demand
        
fp = FusionPlant(0.1, 10_000, 300)
fp.hydrogen_demand, fp.out, fp.shape

(300, 10000, '.')